In [1]:
import torch


from torch_geometric.nn import GCNConv
from torch_geometric.utils import to_dense_adj
from torch.utils.data import DataLoader, random_split

In [2]:
from helpers import CVFConfigDataset

In [3]:
class VanillaGNNLayer(torch.nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.linear = torch.nn.Linear(dim_in, dim_out, bias=False)

    def forward(self, x, adjacency):
        x = self.linear(x)
        # x = torch.sparse.mm(adjacency, x)
        x = torch.matmul(adjacency, x)
        return x

In [4]:
# dataset = CVFConfigDataset(
#     "small_graph_test_config_rank_dataset.csv", "small_graph_edge_index.json", 4
# )
# dataset = CVFConfigDataset(
#     "graph_1_config_rank_dataset.csv", "graph_1_edge_index.json", 10
# )
# dataset = CVFConfigDataset(
#     "graph_4_config_rank_dataset.csv", "graph_4_edge_index.json", 10
# )
dataset = CVFConfigDataset(
    "graph_5_config_rank_dataset.csv", "graph_5_edge_index.json", 9
)
# train_size = int(0.75 * len(dataset))
# test_size = len(dataset) - train_size
# train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
data_loader = DataLoader(dataset, batch_size=128, shuffle=False)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
# adjacency = to_dense_adj(dataset.edge_index.t().contiguous())[0]
# adjacency += torch.eye(len(adjacency))
# adjacency = adjacency.unsqueeze(0)
# adjacency

# dataset.edge_index

In [7]:
class VanillaGNN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gcn1 = GCNConv(dim_in, dim_h)
        self.gcn2 = GCNConv(dim_h, dim_h)
        # self.gnn1 = VanillaGNN(dim_in, dim_h)
        # self.gnn2 = VanillaGNN(dim_h, dim_h)
        self.out = torch.nn.Linear(dim_h, dim_out)

    # def forward_gnn(self, x, adjacency):
    #     h = self.gnn1(x, adjacency)
    #     h = torch.relu(h)
    #     h = self.gnn2(h, adjacency)
    #     h = torch.relu(h)
    #     h = self.out(h)
    #     # h = torch.sigmoid(h)
    #     h = h.squeeze(-1)
    #     return h

    def forward(self, x, edge_index):
        h = self.gcn1(x, edge_index)
        h = torch.relu(h)
        h = self.gcn2(h, edge_index)
        h = torch.relu(h)
        h = self.out(h)
        # h = h.squeeze(-1)
        h = torch.sigmoid(h)
        return h

    def fit(self, data_loader, epochs):
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=5e-4)
        self.train()
        edge_index = dataset.edge_index.t().to(device)
        for epoch in range(1, epochs + 1):
            avg_loss = 0
            count = 0
            for batch in data_loader:
                x = batch[0].to(device)
                y = batch[1].to(device)
                y = y.reshape(-1, 1)
                # print(x)
                # print(y)
                optimizer.zero_grad()
                out = self(x, edge_index)
                # out = self(x, adjacency)
                # print("output", out, "y", y)
                loss = criterion(out, y)
                avg_loss += loss
                count += 1
                loss.backward()
                optimizer.step()

            print("Loss:", avg_loss / count)


In [ ]:
gnn = VanillaGNN(dataset.num_classes, 128, 1).to(device)
print(gnn)

gnn.fit(data_loader, epochs=10)

VanillaGNN(
  (gcn1): GCNConv(9, 64)
  (gcn2): GCNConv(64, 64)
  (out): Linear(in_features=64, out_features=1, bias=True)
)
Loss: tensor(2.0201, device='cuda:0', grad_fn=<DivBackward0>)
Loss: tensor(1.9919, device='cuda:0', grad_fn=<DivBackward0>)
Loss: tensor(1.9922, device='cuda:0', grad_fn=<DivBackward0>)
Loss: tensor(1.9971, device='cuda:0', grad_fn=<DivBackward0>)
Loss: tensor(1.9931, device='cuda:0', grad_fn=<DivBackward0>)
Loss: tensor(1.9940, device='cuda:0', grad_fn=<DivBackward0>)
Loss: tensor(1.9931, device='cuda:0', grad_fn=<DivBackward0>)
Loss: tensor(1.9931, device='cuda:0', grad_fn=<DivBackward0>)
Loss: tensor(1.9987, device='cuda:0', grad_fn=<DivBackward0>)
Loss: tensor(1.9929, device='cuda:0', grad_fn=<DivBackward0>)


In [9]:
torch.set_printoptions(profile="full")

total_matched = 0
edge_index = dataset.edge_index.t().to(device)

for batch in data_loader:
    x = batch[0].to(device)
    y = batch[1].to(device)
    y = y.reshape(-1, 1)
    predicted = gnn(x, edge_index)
    predicted = predicted.argmax(dim=1)
    matched = (predicted == y).sum().item()
    total_matched += matched
    # print("y", y, "predicted", predicted, "Matched", matched)

print("Total matched", total_matched, "| Accuracy", round(total_matched/len(dataset), 4) * 100, "%")

Total matched 155185 | Accuracy 29.2 %


In [10]:
# gnn.fit(data_loader, epochs=25)